A minimum cost maximum flow of a network (G= (V, E)) is a maximum flow with the smallest possible cost. This problem combines maximum flow (getting as much flow as possible from the source to the sink) with shortest path (reaching from the source to the sink with minimum cost).

Note that in this alogrithm the shortest path is defined by edge
cost (weights), not edge capacity


For the unit capacity graph case, we assume that all arcs have unit capacity and that there are no
negative cost arcs. Therefore, the value of any flow in the cycle must be less than or equal to n.
Given that each augmenting path increases the value of the flow by 1, at most n augmentation steps
will suffice in finding the MCF

In [2]:
import numpy as np
from random import random, seed, randint
from copy import deepcopy
from collections import defaultdict, deque

In [3]:
class MinCost:

  def __init__(self, n, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph
        self.weight = {}  # Dictionary to store weights for arcs
        self.capacity = {}
        self.n = n
        self.adj = defaultdict(list)

  def _dict_to_matrix(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to its adjacency matrix.
        """
        n = len(_dict)
        vertices = [*_dict.keys()]
        matrix = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (vertices.index(u), vertices.index(v))
            for u, row in _dict.items() for v in row
        ]:
            matrix[u][v] += 1
        return matrix

    # Modyfying graphs
  def add_vertex(self, vertex):
        """
        Adds a new vertex to the graph.
        """
        if vertex not in self.graph:
            self.graph[vertex] = []

  def add_arc_wc(self, u, v, weight=None, capacity=None):
        """
        Adds a directed arc to the graph with optional weight and capacity.
        """
        self.graph.setdefault(u, []).append(v)
        self.adj[u].append(v)
        self.adj[v]  # Ensure v exists in adj
        if weight is not None:
            self.weight[(u, v)] = weight
        if capacity is not None:
            self.capacity[(u, v)] = capacity
            self.capacity[(v, u)] = 0  # Backward edge with 0 capacity for residual graph


  def bellman_ford(self, source, sink):
        """
        Bellman-Ford algorithm to find the shortest path in terms of weights.
        Returns distances and the parent map.
        """
        dist = [float('inf')] * self.n
        parent = [-1] * self.n
        in_queue = [False] * self.n
        dist[source] = 0
        queue = deque([source])
        in_queue[source] = True

        while queue:
            u = queue.popleft()
            in_queue[u] = False
            for v in self.adj[u]:
                if self.capacity.get((u, v), 0) > 0 and dist[v] > dist[u] + self.weight.get((u, v), 0):
                    dist[v] = dist[u] + self.weight.get((u, v), 0)
                    parent[v] = u
                    if not in_queue[v]:
                        queue.append(v)
                        in_queue[v] = True

        return dist, parent

  def min_cost_max_flow(self, source, sink):
        """
        Finds the minimum cost maximum flow in the graph.
        Returns the total flow and cost.
        """
        flow = 0
        cost = 0
        while True:
            dist, parent = self.bellman_ford(source, sink)
            if dist[sink] == float('inf'):
                break

            # Find the maximum flow through the path found.
            increment_flow = float('inf')
            v = sink
            while v != source:
                u = parent[v]
                increment_flow = min(increment_flow, self.capacity[(u, v)])
                v = u

            # Update capacities and cost.
            v = sink
            while v != source:
                u = parent[v]
                self.capacity[(u, v)] -= increment_flow
                self.capacity[(v, u)] += increment_flow
                cost += increment_flow * self.weight[(u, v)]
                v = u

            flow += increment_flow

        return flow, cost


  def min_cost_flow_given_size(self, source, sink, target_flow):
      """
      Finds the minimum cost for a given flow size in the graph.
      If the required flow cannot be achieved, raises an exception.
      Returns the total flow achieved and its cost.
      """
      flow = 0
      cost = 0

      while flow < target_flow:

        dist, parent = self.bellman_ford(source, sink)
        if dist[sink] == float('inf'):
            raise ValueError("Target flow cannot be achieved: not enough capacity.")


        increment_flow = target_flow - flow
        v = sink
        while v != source:
            u = parent[v]
            increment_flow = min(increment_flow, self.capacity[(u, v)])
            v = u


        v = sink
        while v != source:
            u = parent[v]
            self.capacity[(u, v)] -= increment_flow
            self.capacity[(v, u)] += increment_flow
            cost += increment_flow * self.weight[(u, v)]
            v = u


        flow += increment_flow

      return flow, cost

###Example of Minimum Cost Maximum Flow

In [4]:
# Example usage:
mcmf = MinCost(5)
mcmf.add_vertex(0)
mcmf.add_vertex(1)
mcmf.add_vertex(2)
mcmf.add_vertex(3)
mcmf.add_vertex(4)

mcmf.add_arc_wc(0,1, 5 ,6)
mcmf.add_arc_wc(0, 2, 3, 5)
mcmf.add_arc_wc(1,2,3,4)
mcmf.add_arc_wc(1,3,5,2)
mcmf.add_arc_wc(2,3,6,3)
mcmf.add_arc_wc(2,4,4,5)
mcmf.add_arc_wc(3,4,1,2)
mcmf.add_arc_wc(4,1,3,3)

In [5]:
flow, cost = mcmf.min_cost_max_flow(0, 4)
print(f"Maximum Flow: {flow}, Minimum Cost: {cost}")

Maximum Flow: 7, Minimum Cost: 57


In [137]:
print(mcmf.weight)
print(mcmf.capacity)

{(0, 1): 5, (0, 2): 3, (1, 2): 3, (1, 3): 5, (2, 3): 6, (2, 4): 4, (3, 4): 1, (4, 1): 3}
{(0, 1): 2, (1, 0): 4, (0, 2): 0, (2, 0): 5, (1, 2): 0, (2, 1): 4, (1, 3): 2, (3, 1): 0, (2, 3): 3, (3, 2): 0, (2, 4): 5, (4, 2): 0, (3, 4): 2, (4, 3): 0, (4, 1): 3, (1, 4): 0}


###Example of Minimum Cost of a given size

In [6]:
#Given size

g = MinCost(4)
g.add_arc_wc(0, 1, weight=2, capacity=4)
g.add_arc_wc(0, 2, weight=2, capacity=7)
g.add_arc_wc(1, 2, weight=1, capacity=6)
g.add_arc_wc(1, 3, weight=3, capacity=8)
g.add_arc_wc(2, 3, weight=1, capacity=5)


In [8]:
flow, cost = g.min_cost_flow_given_size(0, 3, target_flow=4)
print(f"Achieved Flow: {flow}, Total Cost: {cost}")

Achieved Flow: 4, Total Cost: 12
